# QIS Cluster Analysis

## Import Libraries

In [7]:
import os
CWD = os.getcwd()
os.chdir(CWD.replace("\\Cluster Analysis\\scripts","") ) 
from EquityHedging.datamanager import data_manager as dm
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as shc
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.offline as py
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster 
import plotly.figure_factory as ff
import plotly.graph_objects as go
from scipy.spatial.distance import pdist, squareform
import ipywidgets as widgets
from ipywidgets import interact


## Import Normalized Hedge Metrics

In [ ]:
file_path = dm.EQUITY_HEDGE_DATA+'def_uni_hedge_metrics_more.xlsx'
#normalized_hm = pd.read_excel(dm.EQUITY_HEDGE_DATA+'def_uni_hedge_metrics_more.xlsx', index_col=(0))
normalized_hm = pd.read_excel(dm.QIS_UNIVERSE +'Normalized_Hedge_Metrics.xlsx', index_col=(0))
features = ['Downside Reliability', 'Convexity', 'Cost', 'Decay','Tail Reliability']
normalized_hm = normalized_hm[features]
normalized_hm.head(20)

In [ ]:
normalized_hm = pd.read_excel(dm.QIS_UNIVERSE + 'Normalized_Hedge_Metrics.xlsx', index_col=0)

features = ['Downside Reliability',  'Convexity', 'Cost', 'Decay', 'Tail Reliability']
normalized_hm = normalized_hm[features]

normalized_hm.head(100)

In [ ]:
# Calculate the sum of features for each strategy (row-wise sum)
normalized_hm['Total'] = normalized_hm.sum(axis=1)

normalized_hm.head(100)

# Devang's code

In [ ]:
# Calculate the pairwise distances between strategies
distances = pdist(normalized_hm)

# Convert the pairwise distances to a square distance matrix
square_distances = squareform(distances)

# Perform hierarchical clustering using linkage on the square distance matrix
linked = linkage(distances, 'ward')

num_clusters = 3

clusters = fcluster(linked, t=num_clusters, criterion='maxclust')
normalized_hm['Cluster'] = clusters


# Function to get the two best-performing strategies in each cluster based on the sum of normalized calculated values
def get_two_best_performing_strategies_per_cluster(df):
    # Group by 'Cluster' and find the two best strategies in each cluster based on 'Total'
    best_strategies_df = df.groupby('Cluster').apply(lambda x: x.nlargest(2, 'Total'))
    return best_strategies_df

# Call the function to get the two best-performing strategies in each cluster
two_best_performing_strategies = get_two_best_performing_strategies_per_cluster(normalized_hm)

print("Two Best Performing Strategies in Each Cluster:")
print(two_best_performing_strategies)

# Function to display the two best-performing strategies for a selected feature within the selected cluster
def show_two_best_strategies_in_cluster(selected_cluster, selected_feature):
    selected_strategies = normalized_hm[normalized_hm['Cluster'] == selected_cluster]
    selected_strategies = selected_strategies.nlargest(2, selected_feature)
    print(f"\nCluster {selected_cluster} - Two Best Performing Strategies based on {selected_feature}:")
    print(selected_strategies)

# Create dropdown widgets for each feature
feature_dropdown = widgets.Dropdown(
    options=features,
    description='Select Feature:'
)

# Create dropdown widget for selecting different cluster numbers
cluster_dropdown = widgets.Dropdown(
    options=normalized_hm['Cluster'].unique(),
    description='Select Cluster:',
    value=num_clusters,  # Default value is the total number of clusters
)

# Interact function to display two best-performing strategies based on selected feature and cluster
interact(show_two_best_strategies_in_cluster, selected_cluster=cluster_dropdown, selected_feature=feature_dropdown);

# best_strategies = normalized_hm.groupby('Cluster')['Total'].idxmax()

# # Get the rows corresponding to the best strategies
# best_performing_strategies = normalized_hm.loc[best_strategies]

# print("Best Performing Strategies in Each Cluster:")
# print(best_performing_strategies)

# #  function to display the strategies in the selected cluster
# def show_strategies_in_cluster(selected_cluster):
#     selected_strategies = normalized_hm[normalized_hm['Cluster'] == selected_cluster].index.tolist()
#     print(f"\nCluster {selected_cluster} Strategies:")
#     print(selected_strategies)

# #  a dropdown widget for selecting different cluster numbers
# cluster_dropdown = widgets.Dropdown(
#     options=normalized_hm['Cluster'].unique(),
#     description='Select Cluster:',
#     value=num_clusters,  # Default value is the total number of clusters
# )


# interact(show_strategies_in_cluster, selected_cluster=cluster_dropdown);

In [ ]:
normalized_hm
top_strats= pd.Da
for i in list(normalized_hm.columns):
    

In [ ]:
best_strategies = normalized_hm.groupby('Cluster')['Total'].idxmax()

# Get the rows corresponding to the best strategies
best_performing_strategies = normalized_hm.loc[best_strategies]

print("Best Performing Strategies in Each Cluster:")
print(best_performing_strategies)


In [ ]:

#  function to display the strategies in the selected cluster
def show_strategies_in_cluster(selected_cluster):
    selected_strategies = normalized_hm[normalized_hm['Cluster'] == selected_cluster].index.tolist()
    print(f"\nCluster {selected_cluster} Strategies:")
    print(selected_strategies)

#  a dropdown widget for selecting different cluster numbers
cluster_dropdown = widgets.Dropdown(
    options=normalized_hm['Cluster'].unique(),
    description='Select Cluster:',
     # Default value is the total number of clusters
)


interact(show_strategies_in_cluster, selected_cluster=cluster_dropdown);

# Create Dendrogram

In [ ]:

# Create the dendrogram figure
dendro_fig = ff.create_dendrogram(links, orientation='bottom')

# Add custom hover text to the dendrogram
dendro_fig.update_traces(hovertext=dendro_fig['data'][0]['text'])

# Customize the layout
dendro_fig.update_layout(width=800, height=700, title='Dendrogram')

# Define a callback function for hover events
def hover_callback(trace, points, state):
    ind = points.point_inds[0]
    cluster_num = clusters[ind]
    strategies = normalized_hm[normalized_hm['Cluster'] == cluster_num].index.tolist()
    tooltip = f"Cluster {cluster_num} Strategies: <br>" + "\n".join(strategies)
    trace.set_hovertext(tooltip)
# Add the callback to the dendrogram
dendro_fig.data[0].on_hover(hover_callback)

# Set the x-axis labels to the names of the strategies
layout = dendro_fig.layout
layout.xaxis.ticktext = normalized_hm.index.tolist()


# Show the dendrogram
dendro_fig.show()

## Find what strats are in each cluster
find what strats rank highest in each cluster 
best in each metric
best over all

In [ ]:
fig = plt.figure(figsize=(20, 10))
plt.xticks(rotation=45)
dend = shc.dendrogram(links)

In [ ]:
# Add a callback function to the dendrogram
def on_click(event):
    if event.mouseevent.button == 1:
        ind = event.ind[0]
        cluster_num = clusters[ind]
        strategies = normalized_hm[normalized_hm['Cluster'] == cluster_num].index.tolist()
        print(f"Cluster {cluster_num} Strategies: {strategies}")

fig.canvas.mpl_connect('pick_event', on_click)
plt.show()

## Dendrogram 

## Principal Component Analysis

In [ ]:
pca = PCA(n_components = 2)
X_principal = pca.fit_transform(normalized_hm)

In [ ]:
X_principal = pd.DataFrame(X_principal, index = normalized_hm.index)
X_principal.columns = ['P1', 'P2']

In [ ]:
X_principal

## Elbow Method

In [ ]:
inertia = []
K = range(1,6)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(normalized_hm)
    kmeanModel.fit(normalized_hm)
    inertia.append(kmeanModel.inertia_)
    
    # Plot the elbow
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.show()

## Determine amount of clusters

In [ ]:
# input clusters here
ncluster = 3

In [ ]:
ac2 = AgglomerativeClustering(n_clusters = ncluster, affinity = 'euclidean', linkage = 'ward')
X_principal['Cluster'] = ac2.fit_predict(X_principal)
X_principal2 = X_principal.copy()
X_principal2['Strat'] = list(X_principal.index)
X_principal2

## Agglomerative Clustering 

In [ ]:
ac2 = AgglomerativeClustering(n_clusters = ncluster, affinity = 'euclidean', linkage = 'ward')
fig = px.scatter(X_principal2, x= 'P1',y ='P2', 
                 color = ac2.fit_predict(X_principal), hover_data = ['Strat'])
fig

In [ ]:
ac2 = AgglomerativeClustering(n_clusters = ncluster, affinity = 'euclidean', linkage = 'single')
fig = px.scatter( x= X_principal['P1'],y =X_principal['P2'], color = ac2.fit_predict(X_principal))
fig

In [ ]:
ac2 = AgglomerativeClustering(n_clusters = ncluster, affinity = 'euclidean', linkage = 'complete')
fig = px.scatter( x= X_principal['P1'],y =X_principal['P2'], color = ac2.fit_predict(X_principal))
fig

In [ ]:
ac2 = AgglomerativeClustering(n_clusters = ncluster, affinity = 'euclidean', linkage = 'average')
fig = px.scatter( x= X_principal['P1'],y =X_principal['P2'], color = ac2.fit_predict(X_principal))
fig